In [2]:
import pandas as pd
import numpy as np
from sklearn.datasets import make_classification

# Generate a synthetic dataset
n_samples = 1000  # Number of samples
n_features = 7    # Number of features
n_classes = 2     # Binary classification (0 and 1)

X, y = make_classification(
    n_samples=n_samples,
    n_features=7,    # Total number of features
    n_informative=7, # All features are informative
    n_redundant=0,   # No redundant features
    n_repeated=0,    # No repeated features
    n_classes=2,     # Binary classification
    random_state=42  # Reproducibility
)

# Convert to a DataFrame for better visualization and manipulation
columns = [f"Feature_{i+1}" for i in range(n_features)]
df = pd.DataFrame(X, columns=columns)
df['Quality'] = y  # Add the target column

# Save the dataset as a CSV file
df.to_csv("binary_classification_dataset.csv", index=False)

# Print first few rows
print(df.head())

   Feature_1  Feature_2  Feature_3  Feature_4  Feature_5  Feature_6  \
0   0.177719  -2.681202  -0.847667  -0.817180   0.335193  -1.735526   
1  -1.001741  -1.565734   1.461369   1.511607   0.897442  -3.641585   
2   1.457602  -0.156201   0.901834   1.021681   1.112249   1.511980   
3   0.156688  -4.783146  -0.484698  -0.754672  -2.574577  -0.356063   
4   0.780096  -0.592755   2.002540  -3.219841  -0.585504   2.134278   

   Feature_7  Quality  
0   2.497458        0  
1   1.453947        0  
2   1.318246        1  
3  -0.682549        0  
4   1.464577        0  


In [3]:
from sklearn.preprocessing import MinMaxScaler

df.loc[df["Quality"] == "good", "Quality"] = 1
df.loc[df["Quality"] == "bad", "Quality"] = 0

scaler = MinMaxScaler(feature_range=(0,1))
scaled_x = scaler.fit_transform(df.iloc[:,:7])

x_train = scaled_x[:40]
x_test = scaled_x[850:]

y_train = df.iloc[:40,7].to_numpy(dtype= np.float32)
y_test = df.iloc[850:,7].to_numpy(dtype= np.float32)
import qiskit
import qiskit.circuit
from qiskit.circuit.library import ZZFeatureMap
from qiskit.circuit.library import TwoLocal
from qiskit_machine_learning.optimizers import L_BFGS_B
from qiskit_machine_learning.algorithms.classifiers import VQC
from matplotlib import pyplot as plt

num_features = 7

feature_map = ZZFeatureMap(num_features,1)
ansatz = TwoLocal(num_features, entanglement="linear",rotation_blocks = qiskit.circuit.library.PhaseGate, entanglement_blocks= "cx", reps=12)


objective_func_vals = []
plt.rcParams["figure.figsize"] = (12, 6)

def callback_graph(weights, obj_func_eval):
    objective_func_vals.append(objective_func_vals)
    print(obj_func_eval)

opt = L_BFGS_B(maxiter= 200, iprint = 101)
vqc = VQC(
    feature_map=feature_map,
    ansatz=ansatz,
    optimizer=opt,
    callback=callback_graph
)

vqc.fit(x_train,y_train)
vqc.optimal_params

1.0


In [11]:
optimal_parameters = vqc.weights
optimal_parameters

array([0.21223948, 0.3932829 , 0.80523734, 0.3937974 , 0.88786859,
       0.60114961, 0.95816174, 0.51916431, 0.05301694, 0.94909863,
       0.77787971, 0.53893387, 0.37127862, 0.02499042, 0.30167287,
       0.71171345, 0.81892061, 0.80649293, 0.32816061, 0.08985057,
       0.3403333 , 0.00505408, 0.77636789, 0.98085838, 0.76305705,
       0.59987424, 0.88073588, 0.66046442, 0.62089464, 0.9834085 ,
       0.95612688, 0.98481587, 0.71048373, 0.63560486, 0.13231083,
       0.94472548, 0.19569212, 0.7027944 , 0.86677502, 0.72513854,
       0.02058157, 0.58719247, 0.43507653, 0.65252028, 0.03558521,
       0.50150357, 0.47227578, 0.49153573, 0.51312695, 0.37994333,
       0.79732973, 0.88930607, 0.94533613, 0.19993751, 0.81888176,
       0.07104097, 0.62652359, 0.29089113, 0.05175284, 0.97373225,
       0.11744636, 0.93888485, 0.2195871 , 0.26281874, 0.68813913,
       0.55974089, 0.84913225, 0.62828379, 0.25850927, 0.11032721,
       0.89305973, 0.39812989, 0.83311126, 0.70217894, 0.50443

In [10]:
train_score = vqc.score(x_train, y_train)
test_score = vqc.score(x_test, y_test)
print(train_score)
print(test_score)

0.65
0.5066666666666667
